In [2]:
from os import TMP_MAX
import cv2
import numpy as np


def pyrometricfunc(img1, img2, T_max):
    import math
    import cv2 as cv
    import numpy as np
    # parameters setting
    T_Max = T_max # temperature(K)
    C1 = 3.7419e-16 # first planck's constant(W*m^2)
    C2 = 1.4388e-2 # second planck's constant(m*K)
    Lambda1 = 400 # (nm)
    Lambda2 = 850 # (nm)
    m2nm = 1e9 # m to nm

    G1 = img1 # grey level
    G2 = img2
    # 这些参数暂时标定
    # S_lambda1 = 0.1911*G1 + 2.9188 # spectral sensitivity (Tungsten lamp factor)
    # S_lambda2 = 0.1911*G2 + 2.9188
    S_lambda1 = 1
    S_lambda2 = 1
    Epsilon_lambda1 = 1 # spectral emissivity 
    Epsilon_lambda2 = 1

    # 注意C2和lambda间的单位换算
    Uppart = C2*(1/Lambda2-1/Lambda1)*m2nm
    Downpart = np.log((G1)/(G2)) + math.log(math.pow(Lambda1/Lambda2, 6),math.e) + math.log((S_lambda2)/(S_lambda1), math.e) # 这里为二维数据
    
    T = Uppart / Downpart
    where_are_nan = np.isnan(T)
    where_are_inf = np.isinf(T)
    #nan替换成0,inf替换成nan
    T[where_are_nan] = 300
    T[where_are_inf] = 300

    T_max = np.max(T)
    # if np.max(T)>T_Max:
    #     T_Max = np.max(T)
    #     print("最高温度T=", T_Max)
    # T = cv.normalize(T,None,0,255,cv.NORM_MINMAX)
    T = T/T_Max*255
    T = T.astype(np.uint8)
    # print(T)
    return T, T_max
    # 对T归一化显示
    # 显示模块
    # def fake_color_display(input: T(2-D))

##选择摄像头
videoLeftUp = cv2.VideoCapture(0)
videoRightUp = cv2.VideoCapture(2)
print("videoleft exposure", videoLeftUp.get(15))
print("videoright exposure", videoRightUp.get(15))
n = -13
videoLeftUp.set(cv2.CAP_PROP_EXPOSURE, n)  #设置曝光值 1.0 - 5000  156.0
videoRightUp.set(cv2.CAP_PROP_EXPOSURE, n)
print("videoleft exposure", videoLeftUp.get(15))
print("videoright exposure", videoRightUp.get(15))

# image.set(cv.CAP_PROP_EXPOSURE, -9)  #设置曝光值 1.0 - 5000  156.0


if not videoLeftUp.isOpened():
    print('Cannot open cameraLeft')
    exit()
if not videoRightUp.isOpened():
    print('Cannot open cameraRight')
    exit()
width = (int(videoLeftUp.get(cv2.CAP_PROP_FRAME_WIDTH)))
height = (int(videoLeftUp.get(cv2.CAP_PROP_FRAME_HEIGHT)))

T_max=300
fraLeft = []
fraRight = []
 
while (videoLeftUp.isOpened()):
    retLeftUp, frameLeftUp = videoLeftUp.read()
    retRightUp, frameRightUp = videoRightUp.read()
    if not retLeftUp & retRightUp:
        print("can't receive frame (stream end?).Exiting ...")
        break
    # 转换为灰度图像
    # frameLeftUp = cv2.cvtColor(frameLeftUp,cv2.COLOR_BGR2GRAY)
    frameLeftUp = frameLeftUp[:,:,0] # 选用蓝色图层
    frameRightUp = cv2.cvtColor(frameRightUp,cv2.COLOR_BGR2GRAY)

    frameLeftUp = cv2.resize(frameLeftUp, (int(width), int(height)), interpolation=cv2.INTER_CUBIC)
    frameRightUp = cv2.resize(frameRightUp, (int(width), int(height)), interpolation=cv2.INTER_CUBIC)
    frameRightUp = cv2.flip(frameRightUp, 1) # 对图像进行左右翻转

    # 处理出热度图像
    T, T_max = pyrometricfunc(frameLeftUp, frameRightUp, T_max)

    fraLeft.append(np.max(frameLeftUp))
    meanLeft = sum(fraLeft)/len(fraLeft)
    if(len(fraLeft)>50):
        fraLeft.remove(fraLeft[0])
    fraRight.append(np.max(frameRightUp))
    meanRight = sum(fraRight)/len(fraRight)
    if(len(fraRight)>50):
        fraRight.remove(fraRight[0])

    cv2.putText(frameLeftUp, "maxBlueLevel="+str(np.max(frameLeftUp)), (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
    cv2.putText(frameRightUp, "maxGreyLevel="+str(np.max(frameRightUp)), (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
    cv2.putText(frameLeftUp, "exposure="+str(videoLeftUp.get(cv2.CAP_PROP_EXPOSURE)), (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
    cv2.putText(frameRightUp, "exposure="+str(videoRightUp.get(cv2.CAP_PROP_EXPOSURE)), (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
    cv2.putText(frameLeftUp, "meanBlueLevel="+str(int(meanLeft)), (0, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
    cv2.putText(frameRightUp, "meanBlueLevel="+str(int(meanRight)), (0, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)   
    #显示图像
    frameUp = np.hstack((frameLeftUp, frameRightUp))
    # frameUp = np.hstack((frameLeftUp, frameRightUp))
    # frameDown = np.hstack((frameLeftDown, frameRightDown))
    # frame = np.vstack((frameUp, frameDown))
    cv2.imshow('frame', frameUp)

    cv2.putText(T, "maxTemperature="+str(T_max), (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
    cv2.imshow("temperature", T)
    print(T_max)
    key = cv2.waitKey(10)
    if int(key) == ord('s'):
        cv2.imwrite("hackpic.jpg", frameUp)
        cv2.imwrite("temperature.jpg", T)
        continue
    if int(key) == ord('q'):
        break
 
videoLeftUp.release()
videoRightUp.release()


videoleft exposure -6.0
videoright exposure -6.0
videoleft exposure -6.0
videoright exposure -6.0


d:\Users\lenovo\Anaconda3\envs\school\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in true_divide
d:\Users\lenovo\Anaconda3\envs\school\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide
d:\Users\lenovo\Anaconda3\envs\school\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log


124315.0083650961
6885.752795644375
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.

: 

In [10]:
import cv2
img1 = cv2.imread("image1.png")
# cv2.imshow("img1", img1)
# cv2.waitKey()
# cv2.destroyAllWindows()
# print(img1)
img_1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
# print(img_1.shape)

In [4]:
# -*- coding: utf-8 -*-
import cv2
import openpyxl
def excel_save(data):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = '相机信息'
    ws['A1'] = '类别'
    ws['B1'] = '数据'
    data_excel = []
    for each in data:
        data_excel.append([each, data[each]])
    for each in data_excel:
        ws.append(each)
    wb.save('data_save.xlsx')
#0内置，1外置
cap = cv2.VideoCapture(2)
data = {
        '视频文件的当前位置（播放）以毫秒为单位':cap.get(0),
        '基于以0开始的被捕获或解码的帧索引':cap.get(1),
        '视频文件的相对位置（播放）：0=电影开始，1=影片的结尾。':cap.get(2),
        '在视频流的帧的宽度':cap.get(3),
        '在视频流的帧的高度':cap.get(4),
        '帧速率':cap.get(5),
        '编解码的4字-字符代码':cap.get(6),
        '视频文件中的帧数':cap.get(7),
        '返回对象的格式':cap.get(8),
        '返回后端特定的值，该值指示当前捕获模式':cap.get(9),
        '图像的亮度(仅适用于照相机)':cap.get(10),
        '图像的对比度(仅适用于照相机)':cap.get(11),
        '图像的饱和度(仅适用于照相机)':cap.get(12),
        '色调图像(仅适用于照相机)':cap.get(13),
        '图像增益(仅适用于照相机)#（Gain在摄影中表示白平衡提升）':cap.get(14),
        '曝光(仅适用于照相机)':cap.get(15),
        '指示是否应将图像转换为RGB布尔标志':cap.get(16),
        '× 暂时不支持':cap.get(17),
        '立体摄像机的矫正标注（目前只有DC1394 v.2.x后端支持这个功能）':cap.get(18)
        }
#print(data)
excel_save(data)


In [5]:
import cv2
cv2.__version__


'4.5.3'

In [2]:
import math
C1 = 3.7419e-16 # first planck's constant(W*m^2)
C2 = 1.4388e-2 # second planck's constant(m*K)
Lambda1 = 900 # (nm)
Lambda2 = 550 # (nm)
m2nm = 1e9 # m to nm

# 注意C2和lambda间的单位换算
Uppart = C2*(1/Lambda2-1/Lambda1)*m2nm
Downpart = math.log(math.pow(Lambda1/Lambda2, 6),math.e)# 这里为二维数据
T = Uppart/(Downpart+27)
print(Uppart, Downpart, T)

10173.333333333334 2.9548589105867653 339.6221415597399
